# EDA

In [2]:
import numpy as np
import pandas as pd

import os
import glob

In [3]:
BASE_DIR = os.path.join(os.getcwd(), os.pardir)
DATA_DIR = os.path.join(BASE_DIR, 'data')

TYPE_LABEL = {'clicks':0, 'carts':1, 'orders':2}

## ファイルimport

In [4]:
def read_file_to_cache(f):
    df = pd.read_parquet(f)
    df.ts = (df.ts/1000).astype('int32')
    df['type'] = df['type'].map(TYPE_LABEL).astype('int8')
    return df

In [11]:
%%timeit
# # ================
# # sample_submission
# # ================
# path_sample_submission = os.path.join(BASE_DIR, 'data', 'original_data', 'sample_submission.csv')
# sample_sub = pd.read_csv(path_sample_submission)

# # ================
# # train/test parquet
# # ================
# train_paths = glob.glob(os.path.join(DATA_DIR, 'chunked_data', 'train_parquet', '*'))
# test_paths = glob.glob(os.path.join(DATA_DIR, 'chunked_data', 'test_parquet', '*'))
# # data_cache_train = {}
# # data_cache_test = {}
# # for p in train_paths: data_cache_train[p] = read_file_to_cache(p)
# # for p in test_paths: data_cache_test[p] = read_file_to_cache(p)

# ================
# validation file
# ================
train_val_paths = glob.glob(os.path.join(DATA_DIR, 'validation', 'train_parquet', '*'))
test_val_paths = glob.glob(os.path.join(DATA_DIR, 'validation', 'test_parquet', '*'))
test_label = os.path.join(DATA_DIR, 'validation', 'test_labels.parquet')
validation_train = {}
# validation_test = {}
for p in train_val_paths: validation_train[p] = read_file_to_cache(p)
# for p in test_val_paths: validation_test[p] = read_file_to_cache(p)
# validation_label = pd.read_parquet(test_label)

CPU times: total: 30.8 s
Wall time: 21.5 s


## EDA

In [15]:
# 1sessionあたりのaid(商品), ts(時刻)のcount, nunique(重複)確認
def _check_ct_duplicate(i):
    print(f'file list index: {i}')
    k = train_val_paths[i]
    g = validation_train[k].groupby('session').agg({'aid':['count', 'nunique'], 'ts': ['count', 'nunique']})
    g.columns = ['_'.join(c) for c in g.columns]
    display(g.describe())

_check_ct_duplicate(0)
_check_ct_duplicate(-1)

file list index: 0


,aid_count,aid_nunique,ts_count,ts_nunique
count,110986.000000,110986.000000,110986.000000,110986.000000
mean,42.921783,24.946777,42.921783,42.470113
std,63.751143,35.157395,63.751143,63.040910
min,1.000000,1.000000,1.000000,1.000000
25%,5.000000,4.000000,5.000000,5.000000
50%,16.000000,10.000000,16.000000,16.000000
75%,52.000000,31.000000,52.000000,51.000000
max,495.000000,460.000000,495.000000,495.000000


file list index: -1


,aid_count,aid_nunique,ts_count,ts_nunique
count,110914.000000,110914.000000,110914.000000,110914.000000
mean,5.918027,4.024163,5.918027,5.826776
std,8.903743,5.473617,8.903743,8.635322
min,1.000000,1.000000,1.000000,1.000000
25%,2.000000,1.000000,2.000000,2.000000
50%,3.000000,2.000000,3.000000,3.000000
75%,6.000000,4.000000,6.000000,6.000000
max,246.000000,168.000000,246.000000,245.000000
